In [44]:
import os
import sys
sys.path.append("/home/pervinco/DL-workspace/NLP/Seq2Seq_Translation")

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

import torch
import torch.nn as nn
import pandas as pd

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

from glob import glob
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split

In [15]:
data_dir = "/home/pervinco/Datasets/KORENG"

In [16]:
df = pd.DataFrame(columns = ['원문','번역문'])
files = sorted(glob(f"{data_dir}/*.xlsx"))

for file in files:
    print(file)

/home/pervinco/Datasets/KORENG/1_구어체(1).xlsx
/home/pervinco/Datasets/KORENG/1_구어체(2).xlsx
/home/pervinco/Datasets/KORENG/2_대화체.xlsx
/home/pervinco/Datasets/KORENG/3_문어체_뉴스(2).xlsx
/home/pervinco/Datasets/KORENG/3_문어체_뉴스(3).xlsx
/home/pervinco/Datasets/KORENG/3_문어체_뉴스(4).xlsx
/home/pervinco/Datasets/KORENG/4_문어체_한국문화.xlsx
/home/pervinco/Datasets/KORENG/5_문어체_조례.xlsx
/home/pervinco/Datasets/KORENG/6_문어체_지자체웹사이트.xlsx


In [17]:
for file in files:
    temp = pd.read_excel(file)
    df = pd.concat([df,temp[['원문','번역문']]])

print(df)

                                                       원문  \
0       'Bible Coloring'은 성경의 아름다운 이야기를 체험 할 수 있는 컬러링 ...   
1                                            씨티은행에서 일하세요?   
2                   푸리토의 베스트셀러는 해외에서 입소문만으로 4차 완판을 기록하였다.   
3        11장에서는 예수님이 이번엔 나사로를 무덤에서 불러내어 죽은 자 가운데서 살리셨습니다.   
4          6.5, 7, 8 사이즈가 몇 개나 더 재입고 될지 제게 알려주시면 감사하겠습니다.   
...                                                   ...   
100082  힐튼호텔 건너편에서 회현동주민센터로 내려가는 길에는 칙칙한 회색의 석축옹벽이 이어져...   
100083  힘든 분들이 많이 계시지만 조금이나마 어렵지 않도록 도움이 될 수 있도록 저희가 최...   
100084  힘든 역사 속에서 대한민국을 불과 50여년 만에 빛나는 나라로 만들고 업적을 만들 ...   
100085  힘든 일을 하는 데는 무엇보다 정부가 큰 관심을 갖고 있다는 자부심을 갖도록 해야 한다.   
100086  힘을 합쳐 세계 일류통일강국으로 대한민국을 우뚝 세우는 역할을 이 자리의 모든 지도...   

                                                      번역문  
0       Bible Coloring' is a coloring application that...  
1                             Do you work at a City bank?  
2       PURITO's bestseller, which recorded 4th rough ...  
3       In Chapter 11 Jesus

In [27]:
df_shuffled=df.sample(frac=1).reset_index(drop=True) ## 모든 행을 무작위로 섞어 새로운 데이터프레임 df_shuffled을 생성.
df = df_shuffled[:10000]

In [28]:
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=False)

print('train size: ', len(train_df))
print('valid size: ', len(valid_df))

train size:  8000
valid size:  2000


In [29]:
print(train_df)

                                                     원문  \
0     무장괴한들이 지난 16일 카메룬 북서부 도시 쿰바에 있는 성 아우구스틴 대학 NSO...   
1     일사병은 장시간 고온에 노출돼 열이 체외로 잘 배출되지 못해 체온이 37도에서 40...   
2                       그때의 햇빛과 모습이 정말 따뜻하게 기억 속에 남아있죠.   
3                                       솔직히 이거 진짜 맛있어요.   
4                          모두 함께 이룬 평화가 모든 이를 위한 평화입니다.   
...                                                 ...   
7995  국내 연구진이 몸 속에 들어가는 미세먼지 경로 추적에 성공했는데 코로 마신 미세먼지...   
7996  이종석이 주연으로 참여하는 한중 합작 드라마 <비취연인(翡翠恋人)>는 촬영 소식들이...   
7997                       한국에 왔었을 때 잘해 주지 못해 정말 미안하다고.   
7998                 지금 한국에는 토요일마다 서울 여러 지역에서 촛불집회가 열려.   
7999  ‘도지사 좀 만납시다’는 7월 11일 경기도청 민원실, 18일 북부청 민원실에서 진...   

                                                    번역문  
0     Armed terrorists ambushed the Saint Augustine ...  
1     Sunstroke refers to the condition of being exp...  
2     The sunshine and image are left as warm memory...  
3        To be honest with you, it is really delicious.  
4

In [33]:
en_tokenizer = get_tokenizer('basic_english')
ko_tokenizer = Mecab()

In [36]:
def insert_tokens(arr):
    arr.insert(0, "<sos>")
    arr.append("<eos>")

    return arr

In [49]:
def tokenize(df, ko_tokenizer, en_tokenizer):
    total_ko_tokens = []
    total_en_tokens = []
    for index, row in df.iterrows():
        ko, en = row["원문"], row["번역문"]
        
        ko_tokens = ko_tokenizer.morphs(ko.strip())
        en_tokens = en_tokenizer(en.strip())

        ko_tokens = insert_tokens(ko_tokens)
        en_tokens = insert_tokens(en_tokens)

        total_ko_tokens.append(ko_tokens)
        total_en_tokens.append(en_tokens)

    return total_ko_tokens, total_en_tokens

In [50]:
train_ko_tokens, train_en_tokens = tokenize(train_df, ko_tokenizer, en_tokenizer)
valid_ko_tokens, valid_en_tokens = tokenize(valid_df, ko_tokenizer, en_tokenizer)

In [54]:
print(train_ko_tokens[0])
print(train_en_tokens[0])

['<sos>', '무장', '괴한', '들', '이', '지난', '16', '일', '카메룬', '북서부', '도시', '쿰바', '에', '있', '는', '성', '아우구스틴', '대학', 'NSO', '캠퍼스', '에', '들어가', '학생', '과', '교사', '176', '명', '을', '납치', '했', '다가', '하루', '만', '인', '17', '일', '석방', '했', '다', '.', '<eos>']
['<sos>', 'armed', 'terrorists', 'ambushed', 'the', 'saint', 'augustine', 'university', 'nso', 'campus', 'in', 'kumbo', ',', 'the', 'city', 'in', 'the', 'north', 'west', 'province', 'of', 'cameroon', ',', 'on', 'the', '16th', 'to', 'kidnap', '176', 'students', 'and', 'teachers', 'and', 'released', 'them', 'the', 'next', 'day', ',', 'the', '17th', '.', '<eos>']


In [51]:
def build_vocab(data_tokens):
    return build_vocab_from_iterator(data_tokens, specials=['<unk>', '<sos>', "<eos>"], min_freq=1)

In [55]:
train_ko_vocab = build_vocab(train_ko_tokens)
train_en_vocab = build_vocab(train_en_tokens)

valid_ko_vocab = build_vocab(valid_ko_tokens)
valid_en_vocab = build_vocab(valid_en_tokens)

In [56]:
print(len(train_ko_vocab), len(train_en_vocab))
print(len(valid_ko_vocab), len(valid_en_vocab))

18683 16160
8444 7464
